In [73]:
import os
from dotenv import load_dotenv

env_file_path = ".env"
load_dotenv(dotenv_path=env_file_path, verbose=True, override=True)

open_ai_api_key = os.getenv('OPENAI_API_KEY')

In [74]:
import json
import hashlib

with open('./documents.json', 'rt') as f_in:
    documents_file = json.load(f_in)

documents = []

def generate_id(doc):
    doc_str = json.dumps(doc, sort_keys=True)  # Convert document to a JSON string
    doc_hash = hashlib.md5(doc_str.encode()).hexdigest()  # Generate an MD5 hash
    return doc_hash

for course in documents_file:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        doc_id = generate_id(doc)  # Generate a unique ID for the document
        doc['doc_id'] = doc_id  # Add the unique ID as a doc_id key
        documents.append(doc)

In [75]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'doc_id': 'bae7a31e6abaddb52b4061dcf238fc61'}

In [76]:
from elasticsearch import Elasticsearch

elasticsearch_host = "http://localhost:9200"
es = Elasticsearch(hosts=elasticsearch_host)
es.info()

ObjectApiResponse({'name': '195933e76752', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'id94QsZrRxWmukjWfVa6eg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [77]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "dynamic": "strict",
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "doc_id": {"type": "keyword"}
        }
    }
}

index_name = "zoomcamp-course-questions"

# Delete the existing index if it exists
try:
    es.indices.delete(index=index_name)
    print(f"Deleted index '{index_name}'")
except NotFoundError:
    print(f"Index '{index_name}' does not exist, no need to delete")
except BadRequestError as e:
    print(f"Error deleting index '{index_name}': {e}")

# Create the index with the new settings
try:
    response = es.indices.create(index=index_name, body=index_settings)
    print(f"Created index '{index_name}': {response}")
except BadRequestError as e:
    print(f"Error creating index '{index_name}': {e}")

response

Deleted index 'zoomcamp-course-questions'
Created index 'zoomcamp-course-questions': {'acknowledged': True, 'shards_acknowledged': True, 'index': 'zoomcamp-course-questions'}


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'zoomcamp-course-questions'})

In [78]:
from tqdm.auto import tqdm

for doc in tqdm(documents, ncols = 100):
    doc_id = doc["doc_id"]
    es.index(index=index_name, id=doc_id, document=doc)

es.count(index=index_name)

100%|████████████████████████████████████████████████████████████| 948/948 [00:02<00:00, 398.06it/s]


ObjectApiResponse({'count': 593, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

In [79]:
user_question = "How do I join the course after it has started?"

search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": user_question,
                    "fields": ["question^3", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}

In [80]:
response = es.search(index=index_name, body=search_query)

for hit in response['hits']['hits']:
    doc = hit['_source']
    print(f"Section: {doc['section']}\nQuestion: {doc['question']}\nAnswer: {doc['text']}\n\n")

Section: General course-related questions
Question: Course - Can I still join the course after the start date?
Answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.


Section: General course-related questions
Question: Course - Can I follow the course after it finishes?
Answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.


Section: General course-related questions
Question: Course - What can I do before the course starts?
Answer: You can start by installing and setting up all the dependencies and requirements:
Google cloud account
Google Cloud SDK
Python 3 (installed with Anaconda)
Terra

In [81]:
def retrieve_documents(query, index_name="zoomcamp-course-questions", max_results=5, elasticseacrh_host=Elasticsearch("http://localhost:9200")):
    
    search_query = {
        "size": max_results,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    
    response = es.search(index=index_name, body=search_query)
    documents = [hit['_source'] for hit in response['hits']['hits']]
    return documents

In [82]:
retrieve_documents(query=user_question)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp',
  'doc_id': '60a31bbef930b3d6b127405fcd0b618e'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp',
  'doc_id': 'cb86516adcdcafa29f0758ae6ca28a0b'},
 {'text': 'You can start by installing and setting up